In [1]:
import numpy as np
import pandas as pd
import glob
import os
import joblib
from datetime import datetime, timedelta
import requests
from nordvpn_switcher import initialize_VPN, rotate_VPN
import time
from bs4 import BeautifulSoup
import xgboost as xgb

%run Functions/Best_Deals_Functions.ipynb
%run Functions/Basic_Functions.ipynb

# Preparations

## Load Data

In [2]:
# when implementing in APP, this dataframe should be preloaded upon start of APP
target_folder = "Data/Transformed Data/Dummy Transformed"
offer_data_path = get_most_recent_file(target_folder, n_most_recent_file = 1)

In [3]:
offer_data = pd.read_csv(offer_data_path, low_memory = False)

## Load Model

In [7]:
# set target folder to search the model in
target_folder_results = "Major Pipeline\\Models\\"

# define if you want the most recent file (1), second most recent (2), ...
n_most_recent_file = 1 

# get the input path
input_path_model = get_data_path(target_folder_results)


# load the model
most_recent_model_file = get_most_recent_file(input_path_model, n_most_recent_file)
xgb_model = joblib.load(most_recent_model_file)


In [8]:
trained_model = xgb_model

# Front End Inputs (get from front-end later)

In [9]:
# OBSERVATION SELECTION PARAMETERS
manufacturer = "Mercedes-Benz" # manufacturer (str):  Choose the brand of the car model youre interested in.
car_model = "180" # car_model (str): Choose the exact car model youre interested in.
max_mileage = 200000 # max_mileage (int): Choose the maximum allowed mileage for the cars youre interested in.
initial_approval = 2000 # initial_approval (int): Choose the earliest allowed initial approval for the car youre interested in.
n_days = 80 # n_days (int): Maximum of how many days back the considered data has been scraped. Is used to make this function more efficient, and not to respect to old offers from the scraped data, as it is probable that they are not available online anymore.


# SCRAPING PARAMETERS
change_vpn = True # change_vpn (boolean): Whether the VPN should be initialized and changed or not.
area_input = ['random countries europe 8'] # area_input (string):  Usually ['random countries europe 8']. Input parameter for nord_vpn switcher, which declares from which pool of IPs NordVPN can pick one when changing the IP-adress.
sleep_time = 0.1 # sleep_time (float): Sleeping time between url requests in seconds.

# BEST OFFER PARAMETERS
n_best_offers = 10 # n_best_offers (int):  Declare how many best offers should be shown.

# Get Best Offers

In [10]:
best_offers_df = get_best_offers_in_one_efficiently_xgb(df = offer_data,
                                        manufacturer = manufacturer,
                                        car_model = car_model,
                                        max_mileage = max_mileage,
                                        initial_approval = initial_approval,
                                        n_days = n_days,
                                        sleep_time = sleep_time,
                                        model = trained_model,
                                        n_best_offers = n_best_offers)

Making Price Predictions...
Predictions done! Found 20 observations for Mercedes-Benz 180 and max mileage of 200000km and Initial Approval >= 2000!
Checking Availability and Creating Result DataFrame!
Best Offers DataFrame created with 10 Offers!


# Check Results

In [13]:
best_offers_df[["Image URL", "Manufacturer", "Model", "Offer Price", "Fair Price", "Savings", "URL"]]

,Image URL,Manufacturer,Model,Offer Price,Fair Price,Savings,URL
0,https://prod.pictures.autoscout24.net/listing-...,Mercedes-Benz,180,12000.00€,19832.15€,7832.15€,https://www.autoscout24.de/angebote/mercedes-b...
1,https://prod.pictures.autoscout24.net/listing-...,Mercedes-Benz,180,10000.00€,11931.45€,1931.45€,https://www.autoscout24.de/angebote/mercedes-b...
2,https://prod.pictures.autoscout24.net/listing-...,Mercedes-Benz,180,11000.00€,12020.97€,1020.97€,https://www.autoscout24.de/angebote/mercedes-b...
3,https://prod.pictures.autoscout24.net/listing-...,Mercedes-Benz,180,11500.00€,12211.20€,711.20€,https://www.autoscout24.de/angebote/mercedes-b...
4,https://prod.pictures.autoscout24.net/listing-...,Mercedes-Benz,180,9700.00€,10316.38€,616.38€,https://www.autoscout24.de/angebote/mercedes-b...
5,https://prod.pictures.autoscout24.net/listing-...,Mercedes-Benz,180,17499.00€,17740.64€,241.64€,https://www.autoscout24.de/angebote/mercedes-b...
6,https://prod.pictures.autoscout24.net/listing-...,Mercedes-Benz,180,27800.00€,27735.08€,-64.91€,https://www.autoscout24.de/angebote/mercedes-b...
7,https://prod.pictures.autoscout24.net/listing-...,Mercedes-Benz,180,14500.00€,14237.57€,-262.43€,https://www.autoscout24.de/angebote/mercedes-b...
8,https://prod.pictures.autoscout24.net/listing-...,Mercedes-Benz,180,13500.00€,12999.51€,-500.49€,https://www.autoscout24.de/angebote/mercedes-b...
9,https://prod.pictures.autoscout24.net/listing-...,Mercedes-Benz,180,8000.00€,6877.02€,-1122.98€,https://www.autoscout24.de/angebote/mercedes-b...
